# Representation theory of 2-Sylow subgroups of symmetric groups


This is a collection of code for describing 2-Sylow subgroups of $S_n$ entirely as forests of $n$. The following code implements the bijection, and calculates character values, class sizes, etc.

## 1. Calculating the Binary Digits of an integer

The binary digits of $n$ is a list $[k_l,k_{l-1},\dots,k_1]$ of strictly decreasing integers such that $n=2^{k_l}+2^{k_{l-1}}+\dots+2^{k_1}$

The function gen_bin_exp(n) returns the binary digits of $n$.


In [3]:
def gen_bin_exp(n):
    """
    Returns the position(starting with 0) of the nonzero digits in the binary expansion of the argument.

    Keyword arguments:
    n -- Positive integer

    """
    L=[]
    i=1
    while n>0:
        if n%(2**i)!=0:         #if the $(i-1)$th position is a 1
            L.append(i-1)         #then record this position in the list
            n=(n-n%(2**i))         #after this the least significant possible nonzero digit will be in the $i$th place
        i+=1
    L.reverse()         #rearrange in descending order
    return L


#### Example:

In [4]:
n=7
gen_bin_exp(n)


[2, 1, 0]

## 2. Generating the forests of $n$
Conjugacy classes and irreducible representations of $H_k$ are both equinumerous with 1-2 binary trees of height $k$.

Lists are used to represent binary trees. A list corresponding to a tree of height $k$ is defined recursively as a list comprising one or two binary trees of height $k-1$. The trivial representation of $H_0$ is denoted by $[[]]$.

A forest of $n$ is defined as a list comprising binary trees of heights corresponding to the binary digits of $n$.

In [10]:
import itertools

def gen_forest(m):
    """
    Iterator returning a forest of size $m$ as a list of binary trees of heights corresponding to the binary digits of $m$.

    Keyword arguments:
    m -- the size of the forest
    """
    def gen_bin_tree(k):
        if k==0:
            yield [[]]         # Yield the trivial tree
        elif k==1:
            yield [[[]]]         #Yield the negative extension
            yield [[[]],[[]]]         #Yield the positive extension 
        else:
            for x in gen_bin_tree(k-1):
                yield [x]         #Yield the negative extension
                yield [x,x]         #Yield the positive extension
            for (x,y) in itertools.combinations(gen_bin_tree(k-1),2):
                yield [x,y]         #Yield trees corresponding to induced representations

    L=gen_bin_exp(m)         #Generate the binary digits of m
    alltrees= [gen_bin_tree(k) for k in L ]         #list of iterables for trees of appropriate height
    for x in itertools.product(*alltrees):
        yield list(x)         #Yield each element in the direct product of iterables as a list


#### Example:

In [11]:
for i,x in enumerate(gen_forest(7)):
    print (i+1),x      #display all forests of 7

1 [[[[[]]]], [[[]]], [[]]]
2 [[[[[]]]], [[[]], [[]]], [[]]]
3 [[[[[]]], [[[]]]], [[[]]], [[]]]
4 [[[[[]]], [[[]]]], [[[]], [[]]], [[]]]
5 [[[[[]], [[]]]], [[[]]], [[]]]
6 [[[[[]], [[]]]], [[[]], [[]]], [[]]]
7 [[[[[]], [[]]], [[[]], [[]]]], [[[]]], [[]]]
8 [[[[[]], [[]]], [[[]], [[]]]], [[[]], [[]]], [[]]]
9 [[[[[]]], [[[]], [[]]]], [[[]]], [[]]]
10 [[[[[]]], [[[]], [[]]]], [[[]], [[]]], [[]]]


Each forest is a list of lists and each element is a list representing a forest. The heights of the trees correspond to the binary digits of the size of the forest.

### 2.1 From forests to binary digits

Given a value for $n$, the above iterable yields all forests of $n$. Now given a forest we would like to find its size; that is, the integer $n$ such that the forest is a forest of $n$. The following function does this.

In [12]:
def size_of_forest(F):
    """
    Takes as input a forest of some integer n, and returns the binary digits of n as a list. The function counts the longest string of opening parantheses at the start of each tree in the list to find its height.
    
    Keyword arguments:
    F- A forest as a list
    """
    def size_of_tree(T):
        x=-1             #corresponding to the empty set
        while len(T)>0:  #the empty list ends the string of continuous left parantheses
            x=x+1        #counts the left parantheses encountered
            T=T[0]       #iteration condition 
        return x
    return [size_of_tree(T) for T in F]        #binary digits of n



#### Example:

In [13]:
n=7
print "Binary digits of",n,":", gen_bin_exp(n)
for x in gen_forest(n):
    print x, ':', size_of_forest(x)

Binary digits of 7 : [2, 1, 0]
[[[[[]]]], [[[]]], [[]]] : [2, 1, 0]
[[[[[]]]], [[[]], [[]]], [[]]] : [2, 1, 0]
[[[[[]]], [[[]]]], [[[]]], [[]]] : [2, 1, 0]
[[[[[]]], [[[]]]], [[[]], [[]]], [[]]] : [2, 1, 0]
[[[[[]], [[]]]], [[[]]], [[]]] : [2, 1, 0]
[[[[[]], [[]]]], [[[]], [[]]], [[]]] : [2, 1, 0]
[[[[[]], [[]]], [[[]], [[]]]], [[[]]], [[]]] : [2, 1, 0]
[[[[[]], [[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] : [2, 1, 0]
[[[[[]]], [[[]], [[]]]], [[[]]], [[]]] : [2, 1, 0]
[[[[[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] : [2, 1, 0]


The example illustrates that the function size_of_forest(F) accepts a forest(as a list of lists) and outputs a list comprising the binary digits of $n$- the size of the forest $F$. 

## 3. Conjugacy classes and Irreducible representations as forests
Let trees of height $k$ of the kind $[T_1,T_2]$ where $T_1$ and $T_2$ are distinct trees of height $k-1$(distinct elementwise as lists) be called Type I trees.

Similarly let trees of the kind $[T,T]$ be called Type II and trees of the kind $[T]$ be called Type III.

A representation of $H_k$ induced from the tensor product of distinct $H_{k-1}$ characters $\phi_1 \otimes \phi_2$ is called Type I, and positive and negative extensions of the representation $\phi \otimes \phi$ are called Type II and Type III trees respectively. The bijection between trees of height $k$ and irreducible representations of $H_k$ proceeds by mapping a representation of a type to a tree of the same type respecting the bijection for height $k-1$ on each component of the tensor. This defines the bijection unambiguously.

Similarly for conjugacy classes of $H_k$, let classes of the type $(\sigma_1,\sigma_2)^{1}$ (for distinct class representatives $\sigma_1$, $\sigma_2$ of $H_{k-1}$) be called Type I classes; $(\sigma,\sigma)^{1}$ type classes are Type II, and $(Id,\sigma)^{-1}$ are Type III. The bijection is defined in the same way for conjugacy classes as is described above for representations.

The bijection extends to $P_n$ componentwise.

With this we define as a first step a method to compute the conjugacy class size, $c_k()$ using the formula(for $H_k$):
$c_k((\sigma_1,\sigma_2)^{1})=2c_{k-1}(\sigma_1)c_{k-1}(\sigma_2)$

$c_k((\sigma,\sigma)^{1})=c_{k-1}(\sigma)c_{k-1}(\sigma)$

$c_k((Id,\sigma)^{-1})=|H_{k-1}|c_{k-1}(\sigma)$

Which extends to $P_n$ by multiplication componentwise. 

We also define here a function to calculate the size of $H_k$ as $2^{2^k-1}$; and for $P_n$ multiplicatively across its components.


### 3.1 Class sizes and Cardinality

In [14]:
def order_of_P(n=0,L=[]):
    """
    Returns the cardinality of the group P_n. Accepts either the integer or its binary digits as input. One or the other is assumed to be a non-null input
    
    Keyword Arguments:
    L- Binary digits of n
    n- Integer
    """
    if n>0:         #checks if integer is provided as input. If both L and n are non-null, n is accepted as the input
        L=gen_bin_exp(n)
    def order_of_H(k):    #For powers of 2
        return 2**(2**k-1) #according to formula
    x=1
    for y in L:
        x=x* order_of_H(y)  #extension of formula to P_n by multiplication across components
    return x

def class_size(F):
    """
    Returns the size of conjugacy class given under the bijection by a forest F
    
    Keyword Arguments:
    F- A forest denoting a conjugacy class"""
    def class_size_k(T): #For powers of 2
        if T==[[]]:  #The only element of H_0
            return 1
        else:
            if len(T)==1:              #Type III representations
                return order_of_P(L=size_of_forest(T))*class_size_k(T[0])  #size_of_forest passes the binary digits of F as argument to order_of_P
            elif T[0]==T[1]:           #Type II representations  
                return class_size_k(T[0])**2
            else:                      #Type I representations   
                return 2*class_size_k(T[0])*class_size_k(T[1])
    x=1
    for y in F:
        x=x*class_size_k(y)
    return x

#### Example:

In [15]:
n=7
print "order_of_P(",n,"):",order_of_P(n)
print "order_of_P(",gen_bin_exp(n),"):",order_of_P(L=gen_bin_exp(n))
sum1=0
for x in gen_forest(n):
    print x, "of size", class_size(x)
    sum1+=class_size(x)
print "Order of n:", order_of_P(n)
print "Sum of class sizes:", sum1

order_of_P( 7 ): 16
order_of_P( [2, 1, 0] ): 16
[[[[[]]]], [[[]]], [[]]] of size 2
[[[[[]]]], [[[]], [[]]], [[]]] of size 2
[[[[[]]], [[[]]]], [[[]]], [[]]] of size 1
[[[[[]]], [[[]]]], [[[]], [[]]], [[]]] of size 1
[[[[[]], [[]]]], [[[]]], [[]]] of size 2
[[[[[]], [[]]]], [[[]], [[]]], [[]]] of size 2
[[[[[]], [[]]], [[[]], [[]]]], [[[]]], [[]]] of size 1
[[[[[]], [[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] of size 1
[[[[[]]], [[[]], [[]]]], [[[]]], [[]]] of size 2
[[[[[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] of size 2
Order of n: 16
Sum of class sizes: 16


The example shows the two ways of using the function- giving either the integer $n$ as input or its binary digits. The next few lines output the sizes of all classes of $P_n$ for a specified value of $n$. Finally the order of $n$ is compared to the sum of sizes of classes, with equality of these two serving as a check of correctness.

## 4. Character value computations
We implement these functions to provide a recursive method of finding character values. Given forests representing a representation and a conjugacy class, the following function defines the character value of the representation at the conjugacy class.

In [16]:
def charval(for_1,for_2):
    """
    Returns the character value of the irreducible representation expressed as a forest on a conjugacy class also expressed as a forest.
    
    Keyword arguments:
    for_1- forest denoting representation of P_n
    for_2- forest denoting conjugacy class of P_n
    
    """  
    def charval_2k(irrep, conj_class):
        if irrep==[[]]:      #if trivial representation
            return 1
        if len(irrep)==1 or irrep[0]==irrep[1]: #Type II or Type III representations
            if len(conj_class)==1:   #Type III conjugacy classes
                return ((-1)**len(irrep))*charval_2k(irrep[0],conj_class[0])
            else:
                return charval_2k(irrep[0],conj_class[0])*charval_2k(irrep[0],conj_class[1])
        else:                       #Type I representations
            if len(conj_class)==1:  #Type III conjugacy classes
                return 0
            else:
                return charval_2k(irrep[0],conj_class[0])*charval_2k(irrep[1],conj_class[1])+charval_2k(irrep[0],conj_class[1])*charval_2k(irrep[1],conj_class[0])
    #Type I and Type II conjugacy classes        
    
    
    if size_of_forest(for_1)== size_of_forest(for_2):  #if representation and conjugacy class are for the same P_n
        x=1
        for i,s in enumerate(for_1):
            x=x*charval_2k(s,for_2[i]) #the character values of the forest is the product of the character values of the trees
        return x
    else:
        return 0
    
    


#### Example:

In [17]:
n=4
for x in gen_forest(n): #for ever representation of P_n
    print [charval(x,y) for y in gen_forest(n)]      #the row of character values for each representation on all classes

[1, 1, -1, 1, -1]
[-1, 1, 1, 1, -1]
[-1, 1, -1, 1, 1]
[1, 1, 1, 1, 1]
[0, -2, 0, 2, 0]


The example shows the calculation of the character table of $P_n$.

### 4.1 The inner product of characters
Finally, a function for the inner product of characters of $H_k$. We use this to verify that the characters defined by our procedure are all independent(in fact orthonormal).

In [18]:
def inner_prod(rep1, rep2):
    """
    Given two (irreducible) characters, calculates its usual inner product.
    
    Keyword arguments:
    rep1- First representation as a forest
    rep2- Second representation as a forest
    """
    if size_of_forest(rep1)!=size_of_forest(rep2):
        return -1          #fail value if both representations are not for the same P_n
    k= sum([2**x for x in size_of_forest(rep1)])  #value of n
    oP= order_of_P(k)                       #Cardinality of P_n
    return sum([charval(rep1, conj_class)*charval(rep2, conj_class)*class_size(conj_class) for conj_class in gen_forest(k)])/oP #usual inner product formula for real valued characters

#### Example:

In [20]:
n=7
print "<X;X>"
for x in gen_forest(n):
    print "<",x,";",x,">=",inner_prod(x,x)

print "<X;Y>"
for (x,y) in itertools.combinations(gen_forest(n),2):
    print "<",x,y,">=",inner_prod(x,y)


<X;X>
< [[[[[]]]], [[[]]], [[]]] ; [[[[[]]]], [[[]]], [[]]] >= 1
< [[[[[]]]], [[[]], [[]]], [[]]] ; [[[[[]]]], [[[]], [[]]], [[]]] >= 1
< [[[[[]]], [[[]]]], [[[]]], [[]]] ; [[[[[]]], [[[]]]], [[[]]], [[]]] >= 1
< [[[[[]]], [[[]]]], [[[]], [[]]], [[]]] ; [[[[[]]], [[[]]]], [[[]], [[]]], [[]]] >= 1
< [[[[[]], [[]]]], [[[]]], [[]]] ; [[[[[]], [[]]]], [[[]]], [[]]] >= 1
< [[[[[]], [[]]]], [[[]], [[]]], [[]]] ; [[[[[]], [[]]]], [[[]], [[]]], [[]]] >= 1
< [[[[[]], [[]]], [[[]], [[]]]], [[[]]], [[]]] ; [[[[[]], [[]]], [[[]], [[]]]], [[[]]], [[]]] >= 1
< [[[[[]], [[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] ; [[[[[]], [[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] >= 1
< [[[[[]]], [[[]], [[]]]], [[[]]], [[]]] ; [[[[[]]], [[[]], [[]]]], [[[]]], [[]]] >= 1
< [[[[[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] ; [[[[[]]], [[[]], [[]]]], [[[]], [[]]], [[]]] >= 1
<X;Y>
< [[[[[]]]], [[[]]], [[]]] [[[[[]]]], [[[]], [[]]], [[]]] >= 0
< [[[[[]]]], [[[]]], [[]]] [[[[[]]], [[[]]]], [[[]]], [[]]] >= 0
< [[[[[]]]], [[[]]

The first few inner products are of the type $\langle \chi; \chi \rangle$ should all be 1. The remaining, of type $\langle \chi_1; \chi_2 \rangle$ need to be $0$ to verify that we have obtained a (complete) list of mutually nonisomorphic irreducible representations of $P_n$.